## Previsión de falla

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

In [2]:
data_set =pd.read_csv(r'C:\Users\juano\Analítica II\MercadoLibre\WorkSample- Applied Intelligence\full_devices.csv', encoding='ISO-8859-1')  
data_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124494 entries, 0 to 124493
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   date        124494 non-null  object
 1   device      124494 non-null  object
 2   failure     124494 non-null  int64 
 3   attribute1  124494 non-null  int64 
 4   attribute2  124494 non-null  int64 
 5   attribute3  124494 non-null  int64 
 6   attribute4  124494 non-null  int64 
 7   attribute5  124494 non-null  int64 
 8   attribute6  124494 non-null  int64 
 9   attribute7  124494 non-null  int64 
 10  attribute8  124494 non-null  int64 
 11  attribute9  124494 non-null  int64 
dtypes: int64(10), object(2)
memory usage: 11.4+ MB


In [3]:
data_set['date'] = pd.to_datetime(data_set['date'])

In [4]:
data_set

,date,device,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9
0,2015-01-01,S1F01085,0,215630672,56,0,52,6,407438,0,0,7
1,2015-01-01,S1F0166B,0,61370680,0,3,0,6,403174,0,0,0
2,2015-01-01,S1F01E6Y,0,173295968,0,0,0,12,237394,0,0,0
3,2015-01-01,S1F01JE0,0,79694024,0,0,0,6,410186,0,0,0
4,2015-01-01,S1F01R2B,0,135970480,0,0,0,15,313173,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...
124489,2015-11-02,Z1F0MA1S,0,18310224,0,0,0,10,353705,8,8,0
124490,2015-11-02,Z1F0Q8RT,0,172556680,96,107,4,11,332792,0,0,13
124491,2015-11-02,Z1F0QK05,0,19029120,4832,0,0,11,350410,0,0,0
124492,2015-11-02,Z1F0QL3N,0,226953408,0,0,0,12,358980,0,0,0


In [5]:
object_describe=data_set.describe(include=["object"]).T 
object_describe

,count,unique,top,freq
device,124494,1169,Z1F0QLC1,304


In [6]:
numeric_describe=data_set.describe().T 
numeric_describe

,count,mean,min,25%,50%,75%,max,std
date,124494,2015-04-16 05:20:49.737337088,2015-01-01 00:00:00,2015-02-09 00:00:00,2015-03-27 00:00:00,2015-06-17 00:00:00,2015-11-02 00:00:00,NaN
failure,124494.0,0.000851,0.0,0.0,0.0,0.0,1.0,0.029167
attribute1,124494.0,122388103.242124,0.0,61284762.0,122797388.0,183309640.0,244140480.0,70459334.21636
attribute2,124494.0,159.484762,0.0,0.0,0.0,0.0,64968.0,2179.65773
attribute3,124494.0,9.940455,0.0,0.0,0.0,0.0,24929.0,185.747321
attribute4,124494.0,1.74112,0.0,0.0,0.0,0.0,1666.0,22.908507
attribute5,124494.0,14.222669,1.0,8.0,10.0,12.0,98.0,15.943028
attribute6,124494.0,260172.657726,8.0,221452.0,249799.5,310266.0,689161.0,99151.078547
attribute7,124494.0,0.292528,0.0,0.0,0.0,0.0,832.0,7.436924
attribute8,124494.0,0.292528,0.0,0.0,0.0,0.0,832.0,7.436924


In [7]:
failure = pd.crosstab(index=data_set["failure"],  # Make a crosstab
                              columns="count")      # Name the count column

failure 

col_0,count
failure,
0,124388
1,106


In [8]:
#dummificamos el data_set por tener una variable categoria
data_set = pd.get_dummies(data_set, columns=['device'], drop_first=True)

In [17]:
#Excluimos variable a predecir y el campo date
X = data_set.drop(['failure','date'], axis=1)
y = data_set['failure']


In [19]:
#separamos el data set en dos bases para entrenar el modelo
import numpy as np
np.random.seed(77300)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,random_state=109)

In [47]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='not majority', random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

In [48]:
# Chequeemos los resultados
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(99595, 1177)
(24899, 1177)
(99595,)
(24899,)


In [49]:
#normalizamos datos

scaler = StandardScaler()
X_train_res_scaled = scaler.fit_transform(X_train_res)
X_test_scaled = scaler.transform(X_test)


# Entrenar un modelo de Random Forest con class_weight='balanced' ya que como se mencionó anteriormente la base se encuentra desbalanceada en cuanto a los datos que se van a predecir

from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
rf_model.fit(X_train_res_scaled, y_train_res)


RandomForestClassifier(class_weight='balanced', random_state=42)

In [50]:
rf_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
rf_model.fit(X_train_res_scaled, y_train_res)
y_pred = rf_model.predict(X_test_scaled)

In [51]:
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))

print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))

Matriz de confusión:
[[24869    10]
 [   19     1]]

Reporte de clasificación:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     24879
           1       0.09      0.05      0.06        20

    accuracy                           1.00     24899
   macro avg       0.55      0.52      0.53     24899
weighted avg       1.00      1.00      1.00     24899



In [67]:
y_pred = (rf_model.predict_proba(X_test_scaled)[:, 1] > 0.1).astype(int)

In [68]:
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))

print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))

Matriz de confusión:
[[24717   162]
 [   13     7]]

Reporte de clasificación:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     24879
           1       0.04      0.35      0.07        20

    accuracy                           0.99     24899
   macro avg       0.52      0.67      0.54     24899
weighted avg       1.00      0.99      1.00     24899

